In [1]:
from libraries.utility import Utility as mutil
from libraries.db_base import db_base
from libraries.db_ins_fake_data import fake_data_to_db
from libraries.custom_excel_output import custom_excel_output
import pandas as pd
import pandasql as psql
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
import warnings
import time
import os
import pandasql as psql
from IPython.core.display import Markdown as md
warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML, Markdown, Latex
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
        '.end_space { min-height:0px !important; } '
        '.prompt {width: 0px; min-width: 0px; visibility: collapse } '
        '.parent{'
        '    display: grid;'
        '    grid-template-columns: 1fr 1fr;'
        '    column-gap: 5px;'
        '}    '
    '</style>'
))



In [2]:
dbprod = db_base("products")

Database : products, Connection Good: True


In [3]:
prod_info = dbprod.run_query_with_single_df(query_key="get_product_info")
prod_info.head(3)

,id,product_name,product_type
0,1,Christina's Wild Ride,TV Season
1,2,Stacy's Date With Destiny,TV Episode
2,3,Marissa's Great War,Bundle


In [4]:
dbfin = db_base("finance",svr_type='postsql')

Database : finance, Connection Good: True


In [5]:
fin_account_activity = dbfin.run_query_with_single_df(query_key="get_account_activity")
fin_account_activity.head(3)

,id,product_id,customer_id,account_id,amt_usd,post_date,channel_desc,partner_desc,location_name,account_name,account_type
0,1,38,1,774,$19.59,2023-06-13 00:44:15,Home Video Rentals,Matthews and Sons,Houston,Theatrical Income Revenue 524618,Revenue
1,2,494,1,133,$23.74,2023-01-09 02:33:35,Museums and Cultural Institutions,Mcdonald LLC,Houston,Theatrical Income Revenue 110374,Revenue
2,3,440,1,620,$21.63,2023-02-27 18:06:05,Music Publishing,"Thomas, Simmons and Brown",Houston,Interest Income Revenue 318141,Revenue


In [6]:
dbcust = db_base("customers",svr_type='mysql')

Database : customers, Connection Good: True


In [7]:
cust_products = dbcust.run_query_with_single_df(query_key="get_customer_product_history1")
cust_products.head(3)

,id,f_name,l_name,product_id,purchase_dt,expiration_dt,created_dt
0,1,Sean,Black,38,2023-06-14 04:18:04,2023-08-31 00:44:15,2023-11-08 20:31:38
1,1,Sean,Black,38,2023-06-13 00:44:15,2023-06-14 04:18:04,2023-11-08 20:31:38
2,1,Sean,Black,440,2023-02-28 15:39:04,2023-02-28 18:06:05,2023-11-08 20:31:38


In [8]:
sql = '''

select * from cust_products where id = 3

'''
df = psql.sqldf(sql)
df.head()

,id,f_name,l_name,product_id,purchase_dt,expiration_dt,created_dt
0,3,Ryan,Mathews,352,2023-09-12 16:16:00.000000,2023-09-12 22:53:12.000000,2023-11-08 20:31:38.000000
1,3,Ryan,Mathews,352,2023-09-11 22:53:12.000000,2023-09-12 16:16:00.000000,2023-11-08 20:31:38.000000
2,3,Ryan,Mathews,132,2023-07-19 13:54:16.000000,2023-07-31 15:48:49.000000,2023-11-08 20:31:38.000000
3,3,Ryan,Mathews,132,2023-07-18 15:48:49.000000,2023-07-19 13:54:16.000000,2023-11-08 20:31:38.000000
4,3,Ryan,Mathews,496,2022-12-15 19:51:14.000000,2023-01-12 21:13:30.000000,2023-11-08 20:31:38.000000


In [9]:
fin_account_activity.head(1)

,id,product_id,customer_id,account_id,amt_usd,post_date,channel_desc,partner_desc,location_name,account_name,account_type
0,1,38,1,774,$19.59,2023-06-13 00:44:15,Home Video Rentals,Matthews and Sons,Houston,Theatrical Income Revenue 524618,Revenue


In [10]:
sql = '''
select
distinct
    cp.id as customer_id,
    cp.f_name,
    cp.l_name,
    p.product_name,
    p.product_type,
    fa.amt_usd,
    fa.post_date,
    fa.channel_desc,
    fa.partner_desc,
    fa.location_name,
    fa.account_name,
    fa.account_type
from cust_products as cp
join prod_info as p
on 
    p.id = cp.product_id
join fin_account_activity fa
on 
 fa.customer_id = cp.id
 and
 fa.product_id = cp.product_id
'''
df = psql.sqldf(sql)


In [11]:
dbcust = custom_excel_output(current_database="customers",svr_type='mysql')

Database : customers, Connection Good: True


In [12]:
# Clean up the post_date column
df['post_date'] = pd.to_datetime(df['post_date'])
df['post_date'] = df['post_date'].dt.strftime('%Y-%m-%d %H:%M:%S')

u = mutil()
u.mkdir(os.path.join(u.get_this_dir(),"project_data","tempdir"))

display(md("### Example of Joined Data"))

file_name = os.path.join(dbcust.get_this_dir(),"project_data","tempdir","customer_account_activity.xlsx")

df["amt_usd"] = df["amt_usd"].replace('[\$,]', '', regex=True).astype(float)

l_dfs = []
l_dfs.append(df)
dbcust.write_excel_from_dfs(list_of_dfs=l_dfs,file_name=file_name,add_subtotal_on_top=True)
href_tag = dbcust.get_embedded_href_tag_from_image_file(file_name)
dbcust.nukefile(file_name)
display(HTML(href_tag))
display(df.head(30).style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])]))

### Example of Joined Data

fmla_buffer =2


,customer_id,f_name,l_name,product_name,product_type,amt_usd,post_date,channel_desc,partner_desc,location_name,account_name,account_type
0,1,Sean,Black,Jason's Wild Ride,Event,19.590000,2023-06-13 00:44:15,Home Video Rentals,Matthews and Sons,Houston,Theatrical Income Revenue 524618,Revenue
1,1,Sean,Black,Jason's Wild Ride,Event,19.590000,2023-06-14 04:18:04,Airports and Airlines,Rodriguez Inc,Houston,Streaming Cash Revenue 521581,Revenue
2,1,Sean,Black,Barbara's Date in the City,TV Episode,21.630000,2023-02-27 18:06:05,Music Publishing,"Thomas, Simmons and Brown",Houston,Interest Income Revenue 318141,Revenue
3,1,Sean,Black,Barbara's Date in the City,TV Episode,21.630000,2023-02-28 15:39:04,Streaming Aggregators,"Mendoza, Morris and Mcguire",Houston,Sell-Through Income Revenue 540371,Revenue
4,1,Sean,Black,Margaret's Patriotic Duty,Bundle,23.740000,2023-01-09 02:33:35,Museums and Cultural Institutions,Mcdonald LLC,Houston,Theatrical Income Revenue 110374,Revenue
5,1,Sean,Black,Margaret's Patriotic Duty,Bundle,23.740000,2023-01-10 13:58:57,Outdoor Events,"Robinson, Brown and Schultz",Houston,Streaming Cash Revenue 958609,Revenue
6,2,Amanda,Alexander,Heidi's Big Romance,Event,23.840000,2023-06-02 23:39:48,Music Publishing,Hebert-Ross,Houston,Rental Income Revenue 166198,Revenue
7,2,Amanda,Alexander,Heidi's Big Romance,Event,23.840000,2023-06-03 16:28:03,Home Video,Tucker-Mullen,Houston,Interest Income Revenue 715149,Revenue
8,2,Amanda,Alexander,Tonya's Date With Destiny,TV Season,27.420000,2023-05-31 09:46:53,Airports and Airlines,"Matthews, Garrett and Scott",Houston,Interest Income Revenue 247881,Revenue
9,2,Amanda,Alexander,Tonya's Date With Destiny,TV Season,27.420000,2023-06-01 01:20:40,Sports Networks,"Simmons, Adams and Bass",Houston,Theatrical Income Revenue 657773,Revenue
